In [ ]:
import requests
import time

In [ ]:
def request_url(url):
    response = requests.get(url)
    while(response.status_code != 200):
        response = requests.get(url)
        time.sleep(1)
    response = response.json()
    return response

In [ ]:
def get_discover_tracks_id(CLIENT_ID):
    result = set()
    
    url = f'https://api-v2.soundcloud.com/mixed-selections?client_id={CLIENT_ID}&limit=100&linked_partitioning=1'
    while(1):
        response = request_url(url)
        collections = response['collection']    
        for collection in collections:
            items = collection['items']['collection']
            for item in items:
                try:
                    tracks = item['tracks']
                    for track in tracks:
                        result.add(track['id'])
                except:
                    break
        try:
            url = response['next_href']+ '&client_id='+CLIENT_ID
        except:
            break
    return result

In [ ]:
CLIENT_ID='YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79'

result=get_discover_tracks_id(CLIENT_ID)
len(result)

In [ ]:
def get_discover_playlists_id(CLIENT_ID):
    result = []
    temp=set()
    url = f'https://api-v2.soundcloud.com/mixed-selections?client_id={CLIENT_ID}&limit=100&linked_partitioning=1'
    while(1):
        response = request_url(url)
        collections = response['collection']    
        for collection in collections:
            items = collection['items']['collection']
            for item in items:
                try:
                    uri = item['uri']
                    temp.add(uri)
                except:
                    break
        try:
            url = response['next_href']+ '&client_id='+CLIENT_ID
        except:
            break
    new_temps=list(temp)
    for new_temp in new_temps:
        result.append(new_temp[37:])
    return result

In [ ]:
CLIENT_ID='YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79'

result = get_discover_playlists_id(CLIENT_ID)

In [ ]:
def extract_playlist_generals(playlist_id, client_id):
    url = f'https://api-v2.soundcloud.com/playlists/{playlist_id}?client_id={client_id}&limit=9&linked_partitioning=1'
    while True:
        time.sleep(1)
        response = requests.get(url)
        if not response.ok:
            print(f"Failed {response.url}")
            continue
        break

    print(f"Hit {response.url}")
    generals_data = response.json()
    generals_data['user'] = generals_data['user']['id']
    print(generals_data['user'])
    tracks_list = []
    for track in generals_data['tracks']:
        tracks_list.append(track['id'])
    generals_data['tracks'] = tracks_list
    return generals_data

In [ ]:
def playlist_info(playlist_id, client_id):
    reposters_url = f'https://api-v2.soundcloud.com/playlists/{playlist_id}/reposters?client_id={client_id}&limit=9&linked_partitioning=1'
    likers_url = f'https://api-v2.soundcloud.com/playlists/{playlist_id}/likers?client_id={client_id}&limit=9&linked_partitioning=1'
    generals_data = extract_playlist_generals(playlist_id, client_id)
    generals_data['reposters'] = get_id_from_collection(reposters_url, client_id, 100)
    generals_data['likers'] = get_id_from_collection(likers_url, client_id, 100)


In [ ]:
def get_id_from_collection(url, client_id, result_limit):
    results = set()
    full = False
    while True:
        response = requests.get(url)
        if not response.ok:
            print(f"Failed {response.url}")
            continue
        print(f"Hit {response.url}")
        try:
            json_data = response.json()
            collections = json_data['collection']
            if not collections:
                break
            for collection in collections:
                if len(results) < result_limit or result_limit==-1 :
                    results.add(collection['id'])
                else:
                    full = True
                    break
            if full or json_data['next_href'] is None:
                break
            url = json_data['next_href'] + f'&client_id={client_id}'
        except KeyError:
            break
    return results

In [ ]:
def get_comment_id_from_collection(url, client_id, result_limit):
    results = set()
    full = False
    while True:
        response = requests.get(url)
        if not response.ok:
            print(f"Failed {response.url}")
            continue
        print(f"Hit {response.url}")
        try:
            json_data = response.json()
            collections = json_data['collection']
            if not collections:
                break
            for collection in collections:
                if len(results) < result_limit or result_limit==-1:
                    results.add(collection['user']['id'])
                else:
                    full = True
                    break
            if full or json_data['next_href'] is None:
                break
            url = json_data['next_href'] + f'&client_id={client_id}'
        except KeyError:
            break
    return results

In [ ]:
def extract_track_generals(track_id, client_id):
    url = f'https://api-v2.soundcloud.com/tracks/{track_id}?client_id={client_id}&linked_partitioning=1'
    while True:
        time.sleep(1)
        response = requests.get(url)
        if not response.ok:
            print(f"Failed {response.url}")
            continue
        break

    print(f"Hit {response.url}")
    generals_data = response.json()
    generals_data['user'] = generals_data['user']['id']
    print(generals_data['user'])

    return generals_data


In [ ]:
def track_info(track_id, client_id):
    reposters_url = f'https://api-v2.soundcloud.com/tracks/{track_id}/reposters?client_id={client_id}&limit=9&linked_partitioning=1'
    likers_url = f'https://api-v2.soundcloud.com/tracks/{track_id}/likers?client_id={client_id}&limit=9&linked_partitioning=1'
    related_tracks_url = f'https://api-v2.soundcloud.com/tracks/{track_id}/related?client_id={client_id}&limit=10&linked_partitioning=1'
    album_url = f'https://api-v2.soundcloud.com/tracks/{track_id}/albums?&client_id={client_id}&limit=1&linked_partitioning=1'
    playlists_url =f'https://api-v2.soundcloud.com/tracks/{track_id}/playlists_without_albums?client_id={client_id}&limit=10&linked_partitioning=1'
    comments_url = f'https://api-v2.soundcloud.com/tracks/{track_id}/comments?threaded=0&filter_replies=0&client_id={client_id}&limit=20&linked_partitioning=1'
    
    generals_data = extract_track_generals(track_id, client_id)
    generals_data['reposters'] = get_id_from_collection(reposters_url, client_id, 100)
    generals_data['likers'] = get_id_from_collection(likers_url, client_id, 100)
    generals_data['comments'] =get_comment_id_from_collection(comments_url, client_id, 100)
    generals_data['related_tracks'] = get_id_from_collection(related_tracks_url, client_id, 100)
    generals_data['album'] = get_id_from_collection(album_url, client_id, 100)
    generals_data['playlists'] = get_id_from_collection(playlists_url, client_id, 100)
    return generals_data

In [ ]:
client_id='YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79'
track_id='591945735'

generals_data=track_info(track_id, client_id)

In [ ]:
generals_data

In [ ]:
def get_discover(client_id):
    
    tracks = get_discover_tracks_id(client_id)
    playlist = get_discover_playlists_id(client_id)
    
    track_result = []
    playlist_result = []
    
    for track_id in tracks:
        track_result.append(track_info(track_id, client_id))
    for playlist_id in playlists:
        playlist_result.append(playlist_info(playlist_id, client_id))
    return track_result,playlist_result

In [ ]:
def extract_user_generals(user_id, client_id):
    url =f'https://api-v2.soundcloud.com/users/{USER_ID}?client_id={CLIENT_ID}&limit=100&linked_partitioning=1'
    while True:
        time.sleep(1)
        response = requests.get(url)
        if not response.ok:
            print(f"Failed {response.url}")
            continue
        break

    print(f"Hit {response.url}")
    generals_data = response.json()
    generals_data['user'] = generals_data['user']['id']
    print(generals_data['user'])

    return generals_data

In [ ]:
def user_info(USER_ID,CLIENT_ID):
    result = {}
    
    general_url =f'https://api-v2.soundcloud.com/users/{USER_ID}?client_id={CLIENT_ID}&limit=100&linked_partitioning=1'
    feauture_profile_url =f'https://api-v2.soundcloud.com/users/{USER_ID}/featured-profiles?client_id={CLIENT_ID}&limit=10&linked_partitioning=1'
    web_profile_url=f'https://api-v2.soundcloud.com/users/soundcloud:users:{USER_ID}/web-profiles?client_id={CLIENT_ID}'
    spotlight_url=f'https://api-v2.soundcloud.com/users/{USER_ID}/spotlight?client_id={CLIENT_ID}&limit=20&linked_partitioning=1'
    user_tracks_url=f'https://api-v2.soundcloud.com/users/{USER_ID}/tracks?client_id={CLIENT_ID}&limit=20&linked_partitioning=1'
    user_top_tracks_url=f'https://api-v2.soundcloud.com/users/{USER_ID}/toptracks?client_id={CLIENT_ID}&linked_partitioning=1'
    user_albums_url=f'https://api-v2.soundcloud.com/users/{USER_ID}/albums?client_id={CLIENT_ID}&limit=10&linked_partitioning=1'
    user_playlist_without_albums_url=f'https://api-v2.soundcloud.com/users/{USER_ID}/playlists_without_albums?client_id={CLIENT_ID}&limit=10&linked_partitioning=1'
    related_artist_url=f'https://api-v2.soundcloud.com/users/{USER_ID}/relatedartists?client_id={CLIENT_ID}&limit=12&linked_partitioning=1'
    reposts_url=f'https://api-v2.soundcloud.com/stream/users/{USER_ID}/reposts?client_id={CLIENT_ID}&limit=10&linked_partitioning=1'
    likes_url=f'https://api-v2.soundcloud.com/users/{USER_ID}/likes?client_id={CLIENT_ID}&limit=24&linked_partitioning=1'
    followings_url=f'https://api-v2.soundcloud.com/users/{USER_ID}/followings?client_id={CLIENT_ID}&limit=3&linked_partitioning=1'
    followers_url=f'https://api-v2.soundcloud.com/users/{USER_ID}/followers?client_id={CLIENT_ID}&limit=12&linked_partitioning=1'
    
    
    generals_data = extract_user_generals(user_id, client_id)
    generals_data['user_playlist_without_albums'] = get_id_from_collection(user_playlist_without_albums_url, client_id, 100)
    generals_data['followings'] = get_id_from_collection(followings_url, client_id, 100)
    generals_data['followers'] = get_id_from_collection(followers_url, client_id, 100)
    
    return result

In [ ]:
#USER_ID='587430921'


#general_url =f'https://api-v2.soundcloud.com/users/{USER_ID}?client_id={CLIENT_ID}&limit=100&linked_partitioning=1'

#feauture_profile_url =f'https://api-v2.soundcloud.com/users/{USER_ID}/featured-profiles?client_id={CLIENT_ID}&limit=10&linked_partitioning=1'
#web_profile_url=f'https://api-v2.soundcloud.com/users/soundcloud:users:{USER_ID}/web-profiles?client_id={CLIENT_ID}'
#spotlight_url=f'https://api-v2.soundcloud.com/users/{USER_ID}/spotlight?client_id={CLIENT_ID}&limit=20&linked_partitioning=1'
#user_tracks_url=f'https://api-v2.soundcloud.com/users/{USER_ID}/tracks?client_id={CLIENT_ID}&limit=20&linked_partitioning=1'
#user_top_tracks_url=f'https://api-v2.soundcloud.com/users/{USER_ID}/toptracks?client_id={CLIENT_ID}&linked_partitioning=1'
#user_albums_url=f'https://api-v2.soundcloud.com/users/{USER_ID}/albums?client_id={CLIENT_ID}&limit=10&linked_partitioning=1'

#playlist_id
#user_playlist_without_albums=f'https://api-v2.soundcloud.com/users/{USER_ID}/playlists_without_albums?client_id={CLIENT_ID}&limit=10&linked_partitioning=1'

#related_artist_url=f'https://api-v2.soundcloud.com/users/{USER_ID}/relatedartists?client_id={CLIENT_ID}&limit=12&linked_partitioning=1'
#reposts_url=f'https://api-v2.soundcloud.com/stream/users/{USER_ID}/reposts?client_id={CLIENT_ID}&limit=10&linked_partitioning=1'
#likes_url=f'https://api-v2.soundcloud.com/users/{USER_ID}/likes?client_id={CLIENT_ID}&limit=24&linked_partitioning=1'
#following_url=f'https://api-v2.soundcloud.com/users/{USER_ID}/followings?client_id={CLIENT_ID}&limit=3&linked_partitioning=1'
#followers_url=f'https://api-v2.soundcloud.com/users/{USER_ID}/followers?client_id={CLIENT_ID}&limit=12&linked_partitioning=1'

#response = request_url(user_playlist_without_albums)
#response['collection'][0]['id']


#response = request_url(reposts_url)
#response['collection'][0]['track']['id']

#response = request_url(likes_url)
#response['collection'][0]['track']['id']

#response = request_url(following_url)
#response['collection'][0]['id']

#response = request_url(followers_url)
#response['collection'][0]['id']

#response = request_url(feauture_profile_url)
#response['collection'][0]['id']

track_id='591945735'
client_id='FjnXkiGFvyaVIYtXadMm9pqIDawoxzUW'
likers_url = f'https://api-v2.soundcloud.com/tracks/{track_id}/likers?client_id={client_id}&limit=100&linked_partitioning=1'

results = get_id_from_collection(likers_url, client_id, -1)
#for result in results:
ls = list(results)


In [ ]:

for l in ls:
    USER_ID=str(l)
    #web_profile_url=f'https://api-v2.soundcloud.com/users/soundcloud:users:{USER_ID}/web-profiles?client_id={client_id}'
    #spotlight_url=f'https://api-v2.soundcloud.com/users/{USER_ID}/spotlight?client_id={CLIENT_ID}&limit=20&linked_partitioning=1'
    #user_tracks_url=f'https://api-v2.soundcloud.com/users/{USER_ID}/tracks?client_id={CLIENT_ID}&limit=20&linked_partitioning=1'
    #user_top_tracks_url=f'https://api-v2.soundcloud.com/users/{USER_ID}/toptracks?client_id={CLIENT_ID}&linked_partitioning=1'
    #user_albums_url=f'https://api-v2.soundcloud.com/users/{USER_ID}/albums?client_id={CLIENT_ID}&limit=10&linked_partitioning=1'
    related_artist_url=f'https://api-v2.soundcloud.com/users/{USER_ID}/relatedartists?client_id={client_id}&limit=12&linked_partitioning=1'
    response = request_url(user_albums_url)
    
    print(l,':',response)

In [ ]:
#587430921 